In [10]:
# File: fitting.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
# One-time app and asset initialization
import os
import random
import shutil
from datetime import datetime
from frontend import App

app = App.create("batch_runner")

# Clone once
codim_ipc_root = "/tmp/Codim-IPC"
checkout_list = [
    # "Projects/FEMShell/input/dress_knife",
    "Projects/FEMShell/input/multilayer",
    "Projects/FEMShell/input/Rumba_Dancing",
]
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

# Load meshes once
stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

V_dress, F_dress, S_stitch = app.extra.load_CIPC_stitch_mesh(stage_path)
V_body, F_body = app.mesh.load_tri(body_path)

# Load animation frames once
body_dir = os.path.dirname(body_path)
body_frames = []
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if not os.path.exists(path):
        break
    Vf, _ = app.mesh.load_tri(path)
    body_frames.append(Vf)
    frame += 1

def run_experiment(idx, params, param_dict, exp_name=None):
    name = f"scene_{idx}"

    # Build scene using cached geometry
    app.asset.clear()
    app.asset.add.tri("dress", V_dress, F_dress)
    app.asset.add.stitch("glue", S_stitch)
    app.asset.add.tri("body", V_body, F_body)

    scene = app.scene.create(name)
    scene.add("dress").stitch("glue").rotate(-90, "x")

    jitter = 0.01 * random.random()
    body = scene.add("body").at(0, -0.78 + jitter, 0).pin()

    for i, V in enumerate(body_frames):
        body.move_to(V, 0.5 + 0.025 * i)

    fixed = scene.build().report()
    fixed.preview(options={"pin": False})

    session = app.session.create(fixed)
    session.start(params).preview()
    session.stream()
    while not session.finished():
        pass   
    print("session finished")
    # Save logs
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    exp_name = exp_name or f"exp_{timestamp}"
    exp_dir = f"./logs/{exp_name}"
    os.makedirs(exp_dir, exist_ok=True)

    for logname in session.get.log.names():
        values = session.get.log.numbers(logname)
        with open(f"{exp_dir}/{logname}.csv", "w") as f:
            f.write("frame,value\n")
            for frame, val in values:
                f.write(f"{frame},{val}\n")

    with open(f"{exp_dir}/stdout.txt", "w") as f:
        for line in session.get.log.stdout():
            f.write(line + "\n")

    with open(f"{exp_dir}/params.txt", "w") as f:
        for key, value in param_dict.items():
            f.write(f"{key}: {value}\n")

    print(f"Experiment {exp_name} saved to {exp_dir}")
    return session.finished()

In [ ]:
def run_experiment(idx, params, param_dict, exp_name=None):
    name = f"scene_{idx}"
    
    # build scene using cached geometry
    app.asset.clear()
    app.asset.add.tri("dress", V_dress, F_dress)
    app.asset.add.stitch("glue", S_stitch)
    app.asset.add.tri("body", V_body, F_body)

    scene = app.scene.create(name)
    scene.add("dress").stitch("glue").rotate(-90, "x")

    jitter = 0.01 * random.random()
    body = scene.add("body").at(0, -0.78 + jitter, 0).pin()

    for i, V in enumerate(body_frames):
        body.move_to(V, 0.5 + 0.025 * i)

    fixed = scene.build().report()
    fixed.preview(options={"pin": False})

    session = app.session.create(fixed)
    session.start(params).preview()
    session.stream()

    while not session.finished():
        pass

    print("session finished")

    # logging
    timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
    exp_name = exp_name or f"exp_{timestamp}"
    exp_dir = f"./logs/{exp_name}"
    os.makedirs(exp_dir, exist_ok=True)

    for logname in session.get.log.names():
        values = session.get.log.numbers(logname)
        with open(f"{exp_dir}/{logname}.csv", "w") as f:
            f.write("frame,value\n")
            for frame, val in values:
                f.write(f"{frame},{val}\n")

    with open(f"{exp_dir}/stdout.txt", "w") as f:
        for line in session.get.log.stdout():
            f.write(line + "\n")

    with open(f"{exp_dir}/params.txt", "w") as f:
        for key, value in param_dict.items():
            f.write(f"{key}: {value}\n")

    print(f"Experiment {exp_name} saved to {exp_dir}")
    return session.finished()


In [ ]:
# Define experiment configurations
base_experiments = [
    {
        "name": "strain_sweep_0.15",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.8e-3,
         "friction": 0.02,
        
    },
    {
        "name": "strain_sweep_0.2",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.2,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
    {
        "name": "strain_sweep_0.27",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.17,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
    {
        "name": "friction_high",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.08,
        "contact-ghat": 0.5e-3,
        "friction": 0.09,
    },
    {
        "name": "dt_large_0.055",
        "dt": 0.002,
        "dyn_dt_change": 0.055,
        "bend": 20,
        "strain-limit-eps": 0.08,
        "contact-ghat": 0.5e-3,
    },
    {
        "name": "contact_sweep",
        "dt": 0.007,
        "dyn_dt_change": 0.04,
        "bend": 5,
        "strain-limit-eps": 0.02,
        "contact-ghat": 1e-4,
    },
]

contact_experiments = [
    {
        "name": "contact_sweep_0.7e-3",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.7e-3,
        "friction": 0.02,
    },
     {
        "name": "contact_sweep_0.6e-3",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.6e-3,
        "friction": 0.02,
    },
     {
        "name": "contact_sweep_0.5e-3",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.5e-3,
        "friction": 0.02,
    },
]



strain_experiments  = [
    {
        "name": "strain_sweep_0.08",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.08,
        "contact-ghat": 0.8e-3,
         "friction": 0.02,
        
    },
    {
        "name": "strain_sweep_0.06",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.06,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
    {
        "name": "strain_sweep_0.05",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.05,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
    #  {
    #     "name": "strain_sweep_0.3",
    #     "dt": 0.002,
    #     "dyn_dt_change": 0.037,
    #     "bend": 20,
    #     "strain-limit-eps": 0.3,
    #     "contact-ghat": 0.8e-3,
    #     "friction": 0.02,
    # },
]

friction_experiments = [
    {
        "name": "friction_sweep_0.05",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.8e-3,
        "friction": 0.05,
    },
     {
        "name": "friction_sweep_0.07",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.8e-3,
        "friction": 0.07,
    },
     {
        "name": "friction_sweep_0.1",
        "dt": 0.002,
        "dyn_dt_change": 0.037,
        "bend": 20,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.8e-3,
        "friction": 0.1,
    },
]

dt_experiments = [
    {
        "name": "dyn_dt_sweep_0.04",
        "dt": 0.002,
        "dyn_dt_change": 0.04,
        "bend": 20,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.7e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep_0.045",
        "dt": 0.002,
        "dyn_dt_change": 0.045,
        "bend": 20,
        "strain-limit-eps": 0.2,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep_0.05",
        "dt": 0.002,
        "dyn_dt_change": 0.05,
        "bend": 20,
        "strain-limit-eps": 0.2,
        "contact-ghat": 0.8e-3,
        "friction": 0.02,
    },
]

dt2_experiments = [
    {
        "name": "dyn_dt_sweep_0.052",
        "dt": 0.002,
        "dyn_dt_change": 0.52,
        "bend": 30,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.6e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep_0.055",
        "dt": 0.002,
        "dyn_dt_change": 0.055,
        "bend": 30,
        "strain-limit-eps": 0.2,
        "contact-ghat": 0.6e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep_0.58",
        "dt": 0.002,
        "dyn_dt_change": 0.058,
        "bend": 30,
        "strain-limit-eps": 0.2,
        "contact-ghat": 0.6e-3,
        "friction": 0.02,
    },
]
dt3_experiments = [
    {
        "name": "dyn_dt_sweep3_0.06",
        "dt": 0.003,
        "dyn_dt_change": 0.06,
        "bend": 20,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.4e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep3_0.065",
        "dt": 0.004,
        "dyn_dt_change": 0.065,
        "bend": 20,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.5e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep3_0.68",
        "dt": 0.004,
        "dyn_dt_change": 0.068,
        "bend": 20,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.5e-3,
        "friction": 0.02,
    },
]
dt4_experiments = [
    {
        "name": "dyn_dt_sweep3_0.03",
        "dt": 0.004,
        "dyn_dt_change": 0.04,
        "bend": 20,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.5e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep3_0.04",#yess
        "dt": 0.004,
        "dyn_dt_change": 0.04,
        "bend": 20,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.2e-3,
        "friction": 0.02,
    },
     {
        "name": "dyn_dt_sweep3_0.45",#failed!
        "dt": 0.004,
        "dyn_dt_change": 0.045,
        "bend": 20,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.4e-3,
        "friction": 0.02,
    },
]

high_iter =  [
    {
        "name": "high_iter1",
        "dt": 0.004,
        "dyn_dt_change": 0.04,
        "bend": 25,
        "strain-limit-eps": 0.12,
        "contact-ghat": 0.4e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter2",#yess
        "dt": 0.004,
        "dyn_dt_change": 0.044,
        "bend": 25,
        "strain-limit-eps": 0.11,
        "contact-ghat": 0.3e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter3",#ripped at shoulder
        "dt": 0.004,
        "dyn_dt_change": 0.045,
        "bend": 25,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.5e-3,
        "friction": 0.02,
    },
]

high_iter2 =  [
    {
        "name": "high_iter3",
        "dt": 0.004,
        "dyn_dt_change": 0.047,
        "bend": 25,
        "strain-limit-eps": 0.10,
        "contact-ghat": 0.3e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter4",#yess
        "dt": 0.004,
        "dyn_dt_change": 0.049,
        "bend": 25,
        "strain-limit-eps": 0.11,
        "contact-ghat": 0.2e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter5",#ripped at shoulder
        "dt": 0.004,
        "dyn_dt_change": 0.05,
        "bend": 35,
        "strain-limit-eps": 0.099,
        "contact-ghat": 0.3e-3,
        "friction": 0.02,
    },
]

high_iter3 =  [
    {
        "name": "high_iter6",
        "dt": 0.004,
        "dyn_dt_change": 0.052,
        "bend": 35,
        "strain-limit-eps": 0.10,
        "contact-ghat": 0.3e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter7",#yess          # failed! 
        "dt": 0.004,
        "dyn_dt_change": 0.055,
        "bend": 35,
        "strain-limit-eps": 0.11,
        "contact-ghat": 0.2e-3,
        "friction": 0.02,
    },
     {
        "name": "high_iter8",#ripped at shoulder
        "dt": 0.004,
        "dyn_dt_change": 0.057,
        "bend": 35,
        "strain-limit-eps": 0.099,
        "contact-ghat": 0.2e-3,
        "friction": 0.02,
    },
]


high_iter4 = [
    {
        "name": "high_iter8",
        "dt": 0.004,
        "dyn_dt_change": 0.036,
        "bend": 25,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.27e-3,
        "friction": 0.002,
    },

    {
        "name": "high_iter12",  
        "dt": 0.004,
        "dyn_dt_change": 0.04,
        "bend": 25,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.25e-3,
        "friction": 0.01,
    },
    {
        "name": "high_iter9", #fail
        "dt": 0.004,
        "dyn_dt_change": 0.050,
        "bend": 35,
        "strain-limit-eps": 0.5,
        "contact-ghat": 0.25e-3,
        "friction": 0.002,
    },
    {
        "name": "high_iter10",   #fail
        "dt": 0.004,
        "dyn_dt_change": 0.055,
        "bend": 25,
        "strain-limit-eps": 0.09,
        "contact-ghat": 0.4e-3,
        "friction": 0.01,
    },
    {
        "name": "high_iter11", 
        "dt": 0.004,
        "dyn_dt_change": 0.057,
        "bend": 35,
        "strain-limit-eps": 0.099,
        "contact-ghat": 0.2e-3,
        "friction": 0.02,
    },
]

fabric_experiments = [
    {"name": "silk","dt": 0.004, "dyn_dt_change": 0.02, "strain-limit-eps": 0.1, "bend": 1, "area-density": 1000, "contact-ghat": 0.0004, "friction": 0.1},
    {"name": "chiffon","dt": 0.004,  "dyn_dt_change": 0.02,"strain-limit-eps": 0.15, "bend": 0.5, "area-density": 500, "contact-ghat": 0.0003, "friction": 0.05},
]

for idx, fabric in enumerate(fabric_experiments):
    param = app.session.param().set("model-shell", "baraffwitkin") 
    run_experiment(idx, param, fabric, exp_name=f"fabric_{fabric['name']}")




shell_models = ["arap", "baraffwitkin"]

# Run all experiments
for idx, exp in enumerate(high_iter3):
    param = (
        app.session.param()
        .set("fitting")
        .set("dt", exp["dt"])
        .set("frames", 240)
        .set("friction", exp["friction"])
        .set("model-shell", "baraffwitkin")
        .set("contact-ghat", exp["contact-ghat"])
        .set("strain-limit-eps", exp["strain-limit-eps"])
        .set("cg-tol", 1e-6)
        .set("cg-max-iter", 1000000)
        .set("bend", exp["bend"])
        .set("area-density", 5000)
        .set("stitch-stiffness",1.0)
        
    )

    param.dyn("fitting").time(0.35).hold().change(False)
    param.dyn("dt").time(0.35).hold().change(exp["dyn_dt_change"])
    sess = run_experiment(idx, param, exp, exp_name=f"{exp['name']}_baraffwitkin")
    print(sess)

# this is for CI


build: 100%|██████████████████████████| 10/10 [00:00<00:00, 48.34it/s]


#vert,#tri,#pin,#stitch_ind
"43,275","84,159","12,811",791


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8948011100292206), quaterni…

interactive(children=(FloatSlider(value=0.0, description='time', max=3.475, step=0.01), Output()), _dom_classe…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8948011100292206), quaterni…

...

interactive(children=(FloatSlider(value=0.0, description='time', max=3.475, step=0.01), Output()), _dom_classe…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8853197991847992), quaterni…

HTML(value='')

HTML(value='')

session finished
Experiment high_iter5_baraffwitkin saved to ./logs/high_iter5_baraffwitkin
True


In [7]:

# Path to your logs folder
logs_dir = "./logs"  # or whatever path you're saving them in
zip_name = "experiment_logs.zip"

# Make a ZIP file
shutil.make_archive(base_name=zip_name.replace(".zip", ""), format='zip', root_dir=logs_dir)

print(f"✅ Logs zipped successfully: {zip_name}")

✅ Logs zipped successfully: experiment_logs.zip


In [1]:

# One-time app and asset initialization
import shutil
import os
import random


from frontend import App
from datetime import datetime

app = App.create("fitting")

codim_ipc_root = "/tmp/Codim-IPC"
checkout_list = [
    "Projects/FEMShell/input/multilayer",
    "Projects/FEMShell/input/Rumba_Dancing",
]
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
app.asset.add.tri("dress", V, F)
app.asset.add.stitch("glue", S)

V, F = app.mesh.load_tri(body_path)
app.asset.add.tri("body", V, F)

scene = app.scene.create()
scene.add("dress").stitch("glue").rotate(-90, "x")

jitter = 0.01 * random.random()
body = scene.add("body").at(0, -0.78 + jitter, 0).pin()
body_dir = os.path.join(codim_ipc_root, checkout_list[1])
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if os.path.exists(path):
        V, _ = app.mesh.load_tri(path)
        body.move_to(V, 0.5 + 0.025 * frame)
        frame += 1
    else:
        break

fixed = scene.build().report()
fixed.preview(options={"pin": False})

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


build: 100%|██████████████████████████| 10/10 [00:00<00:00, 13.73it/s]


#vert,#tri,#pin,#stitch_ind
"43,275","84,159","12,811",791


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.888229250907898), quaternio…

interactive(children=(FloatSlider(value=0.0, description='time', max=3.5, step=0.01), Output()), _dom_classes=…

In [15]:
# this is for multilayer dress
param = (
    app.session.param()
    .set("fitting")
    .set("dt", 0.003)
    .set("frames", 240)
    .set("friction", 0.02)
    .set("model-shell", "baraffwitkin")
    .set("contact-ghat", 0.8e-3) # 0.5e3  parameter controls the activation threshold of contact. Lower values increase contact density:
    .set("strain-limit-eps", 0.2) #0.1 Make it smaller to strongly limit how much the cloth can stretch
    .set("cg-tol", 0.1e-5)
    .set("cg-max-iter",500000)
    .set("bend",25)
    .set("area-density", 10000)
)
param.dyn("fitting").time(0.35).hold().change(False)
param.dyn("dt").time(0.35).hold().change(0.04)   #since 0.032 always converges but 0.034 fails 

session = app.session.create(fixed)
session.start(param).preview()
session.stream()

# param = (
#     app.session.param()
#     .set("fitting")  # Enables automatic fitting of the simulation to the initial state.
#     .set("dt", 0.007)  # [Default: 0.01] Time step size. Smaller values increase accuracy but require more computation.
#     .set("frames", 240)  # [Default: 240] Number of frames to simulate. Determines the total simulation duration.
#     .set("friction", 0.0)  # [Default: 0.0] Coefficient of friction between contacting surfaces. Affects sliding behavior.
#     .set("model-shell", "baraffwitkin")  # [Default: "arap"] Shell model used. Options: "arap" (As-Rigid-As-Possible), "baraffwitkin". Influences deformation behavior.
#     .set("contact-ghat", 0.4e-3)  # [Default: 0.0004] Contact stiffness parameter. Higher values make contact constraints stiffer.
#     .set("strain-limit-eps", 0.09)  # [Default: 0.01] Strain limit for the material. Lower values make the material stiffer.
#     .set("bend", 3)  # [Default: 1] Bending stiffness multiplier. Higher values increase resistance to bending.
#     #.set("area-density", 1000)  # [Default: 1000] Mass per unit area of the shell. Affects inertia and dynamic response.
#     .set("cg-max-iter", 1000000)  # [Default: 1000] Maximum number of iterations for the Conjugate Gradient solver. Higher values allow for more precise solutions.
#     .set("cg-tol", 1e-5)  # [Default: 1e-5] Tolerance for the Conjugate Gradient solver. Lower values increase solution accuracy but may require more iterations.
# )


# param = (
#     app.session.param()
#     .set("fitting")
#     .set("dt", 1e-3)
#     .set("frames", 240)
#     .set("friction", 0.0)
#     .set("model-shell", "arap")
#     .set("contact-ghat", 0.5e-3)
#     .set("strain-limit-eps", 0.1)
# )
# param.dyn("fitting").time(0.15).hold().change(False)
# param.dyn("dt").time(0.15).hold().change(0.01)

# session = app.session.create(fixed)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.888229250907898), quaternio…

HTML(value='')

HTML(value='')

In [ ]:
session.start(param).preview()
session.stream()

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()

In [ ]:
high_iter2 = [
    # {
    #     "name": "high_iter8", #fail
    #     "dt": 0.004,
    #     "dyn_dt_change": 0.042,
    #     "bend": 35,
    #     "strain-limit-eps": 0.1,
    #     "contact-ghat": 0.13e-3,
    #     "friction": 0.002,
    # },
    {
        "name": "high_iter8", #fail
        "dt": 0.004,
        "dyn_dt_change": 0.036,
        "bend": 25,
        "strain-limit-eps": 0.1,
        "contact-ghat": 0.27e-3,
        "friction": 0.002,
    },

    {
        "name": "high_iter12",   #fail
        "dt": 0.004,
        "dyn_dt_change": 0.04,
        "bend": 25,
        "strain-limit-eps": 0.15,
        "contact-ghat": 0.25e-3,
        "friction": 0.01,
    },
    # {
    #     "name": "high_iter11", 
    #     "dt": 0.004,
    #     "dyn_dt_change": 0.057,
    #     "bend": 35,
    #     "strain-limit-eps": 0.099,
    #     "contact-ghat": 0.2e-3,
    #     "friction": 0.02,
    # },
]

fabric_experiments = [
    # {"name": "silk","dt": 0.003, "dyn_dt_change": 0.04, "strain-limit-eps": 0.1, "bend": 1, "area-density": 1000, "contact-ghat": 0.0004, "friction": 0.02},
    # {"name": "chiffon","dt": 0.003,  "dyn_dt_change": 0.04,"strain-limit-eps": 0.15, "bend": 0.5, "area-density": 500, "contact-ghat": 0.0003, "friction": 0.005},
    {"name": "silk_0.052","dt": 0.003, "dyn_dt_change": 0.052, "strain-limit-eps": 0.1, "bend": 1, "area-density": 1000, "contact-ghat": 0.0004, "friction": 0.02},
    {"name": "chiffon_0.052","dt": 0.003,  "dyn_dt_change": 0.052,"strain-limit-eps": 0.15, "bend": 0.5, "area-density": 500, "contact-ghat": 0.0003, "friction": 0.005},
    {"name": "silk_0.055","dt": 0.003, "dyn_dt_change": 0.055, "strain-limit-eps": 0.1, "bend": 1, "area-density": 1000, "contact-ghat": 0.0004, "friction": 0.02},
    {"name": "chiffon_0.055","dt": 0.003,  "dyn_dt_change": 0.055,"strain-limit-eps": 0.15, "bend": 0.5, "area-density": 500, "contact-ghat": 0.0003, "friction": 0.005},
    {"name": "silk_0.057","dt": 0.003, "dyn_dt_change": 0.057, "strain-limit-eps": 0.1, "bend": 1, "area-density": 1000, "contact-ghat": 0.0004, "friction": 0.02},
    {"name": "chiffon_0.057","dt": 0.003,  "dyn_dt_change": 0.057,"strain-limit-eps": 0.15, "bend": 0.5, "area-density": 500, "contact-ghat": 0.0003, "friction": 0.005},


]
#silk: ran pretty slow but no high convergence. did look like silk. 


# Run all experiments
for idx, exp in enumerate(fabric_experiments):
    try:
        param = (
            app.session.param()
            .set("fitting")
            .set("dt", exp["dt"])
            .set("frames", 240)
            .set("friction", exp["friction"])
            .set("model-shell", "baraffwitkin")
            .set("contact-ghat", exp["contact-ghat"])
            .set("strain-limit-eps", exp["strain-limit-eps"])
            .set("cg-tol", 1e-6)
            .set("cg-max-iter", 1000000)
            .set("bend", exp["bend"])
            .set("area-density",exp["area-density"])
            .set("stitch-stiffness", 1.0)
            # .set("contact-offset",1e-1)
        )

        param.dyn("fitting").time(0.35).hold().change(False)
        param.dyn("dt").time(0.35).hold().change(exp["dyn_dt_change"])
        sess = run_experiment(idx, param, exp, exp_name=f"fabric_{exp['name']}")
        print(sess)

    except Exception as e:
        print(f"⚠️ Experiment {exp['name']} failed: {e}")
        continue